# Automatic Tool Annotation for Surgical Workflow Analysis


<b> Setup steps </b>

pip install tqdm

pip install imageio

pip install moviepy

pip install fbpca


pip uninstall -y pillow

CC="cc -mavx2"
pip install -U --force-reinstall pillow-simd


<b> Outstanding Issues </b>

1. RGB - to be solved. 

   1.a Do Robust PCA on R G and B channels separately and create final RBG M and S images - To be done
   
   1.b Try normalizing R G and B channels - To be done
   
   1.c Used Gray PCA to identify pixel mask to get Region of Interest from RGB - Done
   
   1.d Any other idea.. ??
   


2. resizing - solved


3. memory - solved

4. Tuning parameters

    4.a Threshold - 3
    
    4.b k in video - 5 ps
    
    4.c k in pca - 4
    
    4.d maxiteration - 6
    
    4.e clip length  - 10s

Things to Do @ 9/3/2017

1. Ensemble of PCA and FCN to generate masks
2. UNET to build masks, bounding polygons
3. Final Solution

    3.a Multi output CNN to predict class and bounding box
    
    3.b UNET to predict bounding box by class
    
NEED TO REDO. BUG IN VIDEO ITERATOR

In [1]:
import imageio
imageio.plugins.ffmpeg.download()

In [2]:
import moviepy.editor as mpe
from glob import glob
import sys, os
import numpy as np
import scipy
%matplotlib inline
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
from scipy import sparse
from sklearn.utils.extmath import randomized_svd
import fbpca
import bcolz
import utils; reload(utils)
from utils import *
import scipy.signal as sci

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [3]:
# MAX_ITERS = 10
TOL=1e-9
MAX_ITERS=3

In [4]:
def create_data_matrix_from_video(clip, k=5):
    totalFrames = k * int(clip.duration)
    FIRST_TIME = True
    for i in range(totalFrames):
        frame = resize(clip.get_frame(i/float(k)))
        if(FIRST_TIME):
            gray = rgb2gray(frame).flatten()
            rgb = frame.flatten()
            FIRST_TIME = False
        else:
            rgb = np.vstack((rgb,frame.flatten()))
            gray = np.vstack((gray,rgb2gray(frame).flatten()))          
    return gray.T,rgb.T

In [5]:
def select_frames_from_data_matrix(mRGB,MASK,RGB,S, k=5, duration=30):
    totalFrames = int(k * duration)
    mRGBSize = mRGB.shape[1]
    FIRST_TIME = True
    for i in range(totalFrames):
        frame = mRGB[:,i*mRGBSize/totalFrames]
        maskSlice = MASK[:,i*mRGBSize/totalFrames]
        rgbSlice = RGB[:,i*mRGBSize/totalFrames]
        sSlice = S[:,i*mRGBSize/totalFrames]
        if(FIRST_TIME):
            mrgb = frame
            mask = maskSlice
            rgb = rgbSlice
            s = sSlice
            FIRST_TIME = False
        else:
            mrgb = np.vstack((mrgb,frame))
            mask = np.vstack((mask,maskSlice))
            rgb = np.vstack((rgb,rgbSlice))
            s = np.vstack((s,sSlice))
    return mrgb.T,mask.T,rgb.T,s.T

In [6]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [7]:
def resize(frame):
    w1 = frame.shape[0]
    crop = (224-w1)/2
    margin = (224-w1)/2
    cons = np.int(np.median(frame))
    img = frame.transpose(2,0,1).reshape(3,w1,224)
    pads = ((margin,margin),(0,0))
    img_arr = np.ndarray((3,224,224),np.int)
    for i,x in enumerate(img):
        x_p = np.pad(x,pads,'constant',constant_values=cons)
        img_arr[i,:,:] = x_p
    return np.uint8(img_arr).transpose(1,2,0)

In [8]:
def plt_images(M, A,index_array, dims, filename=None):
    f = plt.figure(figsize=(10, 10))
    r = len(index_array)
    pics = r * 2
    for k, i in enumerate(index_array):
        for j, mat in enumerate([M, A]):
            sp = f.add_subplot(r, 2, 2*k + j + 1)
            sp.axis('Off')
            pixels = mat[:,i]
            if isinstance(pixels, scipy.sparse.csr_matrix):
                pixels = pixels.todense()
            plt.imshow(np.reshape(pixels, dims))
    return f

In [9]:
def plots(ims, dims, figsize=(15,20), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims)
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        plt.imshow(np.reshape(ims[i], dims), cmap="gray")

In [10]:
def converged(Z, d_norm):
    err = np.linalg.norm(Z, 'fro') / d_norm
    #print('error: ', err)
    return err < TOL

In [11]:
def shrink(M, tau):
    S = np.abs(M) - tau
    return np.sign(M) * np.where(S>0, S, 0)

In [12]:
def _svd(M, rank): return fbpca.pca(M, k=min(rank, np.min(M.shape)), raw=True)

In [13]:
def norm_op(M): return _svd(M, 1)[1][0]

In [14]:
def svd_reconstruct(M, rank, min_sv):
    u, s, v = _svd(M, rank)
    s -= min_sv
    nnz = (s > 0).sum()
    return np.matmul(np.matmul(u[:,:nnz],np.diag(s[:nnz])) , v[:nnz]), nnz

In [15]:
def pcp(X, maxiter=10, k=10): # refactored
    m, n = X.shape
    trans = m<n
    if trans: X = X.T; m, n = X.shape
        
    lamda = 1/np.sqrt(m)
    op_norm = norm_op(X)
    Y = np.copy(X) / max(op_norm, np.linalg.norm( X, np.inf) / lamda)
    mu = k*1.25/op_norm; mu_bar = mu * 1e7; rho = k * 1.5
    
    d_norm = np.linalg.norm(X, 'fro')
    L = np.zeros_like(X); sv = 1
    
    examples = []
    
    for i in range(maxiter):
        #print("rank sv:", sv)
        X2 = X + Y/mu
        
        # update estimate of Sparse Matrix by "shrinking/truncating": original - low-rank
        S = shrink(X2 - L, lamda/mu)
        
        # update estimate of Low-rank Matrix by doing truncated SVD of rank sv & reconstructing.
        # count of singular values > 1/mu is returned as svp
        L, svp = svd_reconstruct(X2 - S, sv, 1/mu)
        
        # If svp < sv, you are already calculating enough singular values.
        # If not, add 20% (in this case 240) to sv
        sv = svp + (1 if svp < sv else round(0.05*n))
        
        # residual
        Z = X - L - S
        Y += mu*Z; mu *= rho
        
        examples.extend([S[140,:], L[140,:]])
        
        if m > mu_bar: m = mu_bar
        if converged(Z, d_norm): break
    
    if trans: L=L.T; S=S.T
    return L, S, examples

In [16]:
def maskRGB(RGB,S):
    
    totalFrames = RGB.shape[1]  
    threshold = 3
    FILL_VALUE = np.int(np.percentile(RGB,50))
    cfilter = np.reshape(np.array([[0.111111111] * 9]),(3,3))
    FIRST_TIME = True

    for image in range(totalFrames):
        S_conv = sci.convolve2d(np.reshape(S[:,image],(224,224)),cfilter,mode='same')
        S_conv[abs(S_conv)<threshold]=0
        S_conv[abs(S_conv)>=threshold]=1
        mask = S_conv.astype(int)
        rgbImage = np.reshape(RGB[:,image],(224,224,3))
        img = rgbImage.transpose(2,0,1).reshape(3,224,224)
        img_arr = np.ndarray((3,224,224),np.int)
        for i,x in enumerate(img):
            x_p = np.multiply(mask,x)
            img_arr[i,:,:] = x_p
        mask = (mask - 1)*-1

        img_arr2 = np.ndarray((3,224,224),np.int)
        for i,x in enumerate(img_arr):
            x_p = x + mask*FILL_VALUE
            img_arr2[i,:,:] = x_p
        rgbImage = np.uint8(img_arr2).transpose(1,2,0)
        if(FIRST_TIME):
            mRGB = rgbImage.flatten()
            MASK = mask.flatten()
            FIRST_TIME = False
        else:
            mRGB = np.vstack((mRGB,rgbImage.flatten()))
            MASK = np.vstack((MASK,mask.flatten()))
    
    return mRGB.T,MASK.T

In [17]:
def augmentData(path,name):
    #Need to add  code to simultaneously augment all arrays together
    #Need to add code to read and store labels
    #Need to decide whether to overwrite arrays with augmented images or not
    
    mrgb = load_array(path+name+'_mRGB.dat')
    hflip = ImageDataGenerator(horizontal_flip=True)
    vflip = ImageDataGenerator(vertical_flip=True)
    rotate = ImageDataGenerator(rotation_range=90)
    zoom = ImageDataGenerator(zoom_range=(0.8,1.1))

    totalFrames = mrgb.shape[1]
    mrgbAug = np.zeros([totalFrames, 3, 224, 224 ],dtype=np.uint8)

    for frameIndex in range(totalFrames):
        x_train = np.reshape(mrgb[...,frameIndex],(224,224,3))
        x_train = np.transpose(x_train,(2,0,1))
        images_0 = np.zeros([1, 3, 224, 224 ],dtype=np.uint8)
        labels_dummy = np.zeros([1, 1],dtype=np.int8)
        images_0[0] = x_train
        images = np.zeros([1, 3, 224, 224 ],dtype=np.uint8)
        if frameIndex%5==0:
            for x,y in hflip.flow(images_0,labels_dummy,batch_size=1):
                mrgbAug[frameIndex] = x
                break
        if frameIndex%5==1:
            for x,y in vflip.flow(images_0,labels_dummy,batch_size=1):
                mrgbAug[frameIndex] = x
                break
        if frameIndex%5==2:
            for x,y in rotate.flow(images_0,labels_dummy,batch_size=1):
                mrgbAug[frameIndex] = x
                break
        if frameIndex%5==3:
            for x,y in zoom.flow(images_0,labels_dummy,batch_size=1):
                mrgbAug[frameIndex] = x
                break
        if frameIndex%5==4:
            mrgbAug[frameIndex] = x_train
    save_array(path+name+'_mRGBAug.dat',mrgbAug)

In [18]:
def processVideos(path,name):
    l = 1920
    w = 1080
    w1 = int(224.0/l*w)
    video = mpe.VideoFileClip(path+name+'.mp4',target_resolution=(w1,224))
    clipDuration = 30
    totalDuration = video.duration
    countClips = int(math.ceil(totalDuration/clipDuration))
    FIRST_TIME = True
    chunkIndex = 0
    for clipIndex in range(countClips):
        clip = video.subclip(clipIndex*clipDuration,clipIndex*clipDuration+clipDuration)
        M, RGB = create_data_matrix_from_video(clip, k = 29)
        L, S, examples =  pcp(M, maxiter=6, k=4)
        del(L,examples,M)
        MRGB, MASK = maskRGB(RGB,S)
        mRGBChunk,maskChunk,rgbChunk,sChunk = select_frames_from_data_matrix(MRGB,MASK,RGB,S)
        del(MRGB,MASK,RGB,S)
        gc.collect()
        if(FIRST_TIME):
            mRGB = mRGBChunk
            mask = maskChunk
            rgb = rgbChunk
            s = sChunk
            FIRST_TIME = False
        else:
            mRGB = np.hstack((mRGB,mRGBChunk))
            mask = np.hstack((mask,maskChunk))
            rgb = np.hstack((rgb,rgbChunk))
            s = np.hstack((s,sChunk))  
        del(mRGBChunk,maskChunk,rgbChunk,sChunk)
        gc.collect()
        if mRGB.shape[1]>2500:
            save_array('../03_output/'+name+'_mRGB_%d.dat'%chunkIndex,mRGB)
            save_array('../03_output/'+name+'_mask_%d.dat'%chunkIndex,mask)
            save_array('../03_output/'+name+'_s_%d.dat'%chunkIndex,s)
            save_array('../03_output/'+name+'_rgb_%d.dat'%chunkIndex,rgb)
            chunkIndex = chunkIndex + 1
            del(mRGB,mask,s,rgb)
            gc.collect()
            FIRST_TIME = True
    save_array('../03_output/'+name+'_mRGB_%d.dat'%chunkIndex,mRGB)
    save_array('../03_output/'+name+'_mask_%d.dat'%chunkIndex,mask)
    save_array('../03_output/'+name+'_s_%d.dat'%chunkIndex,s)
    save_array('../03_output/'+name+'_rgb_%d.dat'%chunkIndex,rgb)
    del(mRGB,mask,s,rgb)
    gc.collect()

In [ ]:
videos = glob('../01_input/train/*.mp4')        
for i in tqdm(range(len(videos))):
    name = videos[i].split("../01_input/train/")[1].split('.mp4')[0]
    processVideos('../01_input/train/',name)

 38%|███▊      | 3/8 [3:12:08<4:59:38, 3595.65s/it]

In [ ]:
videos = glob('../01_input/test/*.mp4')        
for i in tqdm(range(len(videos))):
    name = videos[i].split("../01_input/test/")[1].split('.mp4')[0]
    processVideos('../01_input/test/',name)

In [ ]:
videos = glob('../01_input/train/*.mp4')        
for i in tqdm(range(len(videos))):
    name = videos[i].split("../01_input/train/")[1].split('.mp4')[0]
    augmentData('../01_input/train/',name)

In [ ]:
videos = glob('../01_input/test/*.mp4')        
for i in tqdm(range(len(videos))):
    name = videos[i].split("../01_input/test/")[1].split('.mp4')[0]
    augmentData('../01_input/train/',name)

# Scrapbook

In [ ]:
p = pd.read_csv('../01_input/labels/train01.csv')

In [ ]:
l = 1920
w = 1080
w1 = int(224.0/l*w)
path='../01_input/train/'
name='train01'
video = mpe.VideoFileClip(path+name+'.mp4',target_resolution=(w1,224))
clipDuration = 10
totalDuration = video.duration
countClips = int(math.ceil(totalDuration/10))
FIRST_TIME = True
for clipIndex in range(countClips):
    clip = video.subclip(clipIndex*clipDuration,clipIndex*clipDuration+clipDuration)
    M, RGB = create_data_matrix_from_video(clip, k = 100)
    L, S, examples =  pcp(M, maxiter=6, k=4)
    del(L,examples,M)
    MRGB, MASK = maskRGB(RGB,S)
    mRGBChunk,maskChunk,rgbChunk,sChunk = select_frames_from_data_matrix(MRGB,MASK,RGB,S)
    if(FIRST_TIME):
        mRGB = mRGBChunk
        mask = maskChunk
        rgb = rgbChunk
        s = sChunk
        FIRST_TIME = False
    else:
        mRGB = np.hstack((mRGB,mRGBChunk))
        mask = np.hstack((mask,maskChunk))
        rgb = np.hstack((rgb,rgbChunk))
        s = np.hstack((s,sChunk))

In [ ]:
countClips = 2

In [ ]:
for clipIndex in range(countClips):
    clip = video.subclip(clipIndex*clipDuration,clipIndex*clipDuration+clipDuration)
    M, RGB = create_data_matrix_from_video(clip, k = 100)
    L, S, examples =  pcp(M, maxiter=6, k=4)
    del(L,examples,M)
    MRGB, MASK = maskRGB(RGB,S)
    mRGBChunk,maskChunk,rgbChunk,sChunk = select_frames_from_data_matrix(MRGB,MASK,RGB,S)
    if(FIRST_TIME):
        mRGB = mRGBChunk
        mask = maskChunk
        rgb = rgbChunk
        s = sChunk
        FIRST_TIME = False
    else:
        mRGB = np.hstack((mRGB,mRGBChunk))
        mask = np.hstack((mask,maskChunk))
        rgb = np.hstack((rgb,rgbChunk))
        s = np.hstack((s,sChunk))


In [ ]:
mRGB.shape, mask.shape , rgb.shape , s.shape

In [ ]:
processVideos('../01_input/train/','train01')

In [ ]:
plt.imshow(np.reshape(mRGB[:,45],(224,224,3))) # k = 100 , 10 sec

In [ ]:
mRGB.shape

In [ ]:
M.shape, S.shape, len(examples)

In [ ]:
RGB = load_array('../03_output/train01_RGB.dat')
S = load_array('../03_output/train01_S.dat')

In [ ]:
mRGB = maskRGB(RGB,S)    

In [ ]:
plt.imshow(np.reshape(mRGB[:,300],(224,224,3)))

In [ ]:
L, S, examples =  pcp(M, maxiter=6, k=10)

In [ ]:
#debug
videos = glob('../01_input/train/*.mp4')   
totalDuration = 0
for i in range(len(videos)):
    name = videos[i].split("../01_input/train/")[1].split('.mp4')[0]
    video = mpe.VideoFileClip('../01_input/train/'+name+'.mp4')
    print int(round(5*500/video.duration)), video.duration, video.size, video.fps, name
    totalDuration += video.duration
print totalDuration

In [ ]:
#debug
#processVideos('../01_input/train/','train19')
#M = np.load('../03_output/train19_M.npy')
#S = np.load('../03_output/train19_S.npy')
dims = (224,224,3)
f = plt_images(M, S, [0, 50, 100], dims)
gc.collect()